In [4]:
import numpy as np
import matplotlib.pyplot as plt

# extract info from data

In [ ]:
path = '../../rltest_data/'

nps_ch1 =
nps_ch2 =
nps_ch6 =

sev_ch1 =
sev_ch2 =
sev_ch6 =

transition_curves = 

In [5]:
! ls ../../rltest_data/

Channel_0_NPS.xy
Channel_0_SEV_Particle.xy
Channel_1_NPS.xy
Channel_1_SEV_Particle.xy
Channel_5_NPS.xy
Channel_5_SEV_Particle.xy
Li1.jpeg
Li2.jpeg
Screenshot_20230221_182807_Samsung Notes.jpg
instructions.rtf
mqtt_tests
photo_2023-02-22 19.24.59.jpeg
photo_2023-02-22 19.25.02.jpeg
rl_tests.xy


# calc detector properties

In [2]:
# Infos
# units are always mK, pJ, pW, Ohm, H, muA, V, s, Hz, mm, keV, g, ...

# things to take from data:

# tau_film and tau_crystal from tau_n = ( 1/tau_film + 1/tau_crystal )^-1
# and tau_film from A_fast = epsilon E / Ce and epsilon = tau_film / (tau_film + tau_crystal)
# this gives tau_film = (1 + beta) tau_n and tau_crystal = (1 + beta) / beta tau_n
# with beta = E / A_fast / Ce - 1

# but we need A_fast in term of temperature

# for this we need the transition curve and the OP at the point where A_fast was measured
# scale the transition curve to the bias current and get derivative at DAC
# get V/phi_0 from Heater bias client: Phi-0 [Volt / Phi-0]: 8.65
# the input coil sensitivity is then 0.15 muA/phi_0
# therefore 8.65 V PH = 0.15 muA PH * der sensitivity range (zehnerpotenz)
# sensitivity range get from squid client - it is * 100

# bias currents were: Li1P 2.8, Li1L 0.84, Li2P 3.0
# V_sets were: Li1P 1.55, Li1L 0.287, Li2P 2.5

# critical temperature
# tes_fluct is the prefactor of the 1/f noise component
# emi is the prefactor of the 50 Hz peak
# Rh, or estimate 

# get NPS, calc expected noise and fit other components

absorber_pars = {
    'material': 'LiAl02',  # 'Si' for light detector 
    'volume': 4000,  # mm^3, for ligh detector 200
    'surface': 1600,  # mm^2, for light detector 840
    'tau_crystal': ...,  # s / mm^3 * mm^2 - multiply with volume absorber (mm^3), divide by surface (mm^2)  # should also scale with sound velocity absorber
           }

tes_pars = {
    'material': 'W',
    'thickness': 0.0002,  # mm  # size "S"
    'surface': 1.02,  # mm^2
    'surface_phonon_collector': 7.6,  # mm^2
    'critical_temperature': ...,  # mK
    'tau_film': ...,  # s / mm^3 * mm^2 - multiply with volume absorber (mm^3), divide by surface interface (TES + collectors) (mm^2)  # should also scale with sound velocity absorber
    'Rt0': 0.110,  # Ohm, 0.1 for Li2 Phonon
    'tes_fluct': ...,  # ?
    
           }

utility_pars = {
    'bath_temperature': 11,  # mK
    'Rh': 10,  # Ohm
    'Rs': 0.035,  # Ohm
    'L': 3.5e-7, # H
    'emi': ...,  # 
            }

# below are constants of the materials

fermi_temp = {'W': 27_000_000, 'Au': 63_900_000, 'Al': 134_900_000}  # mK
debye_temp = {'W': 383_000, 'Au': 162_000, 'Al': 428_000, 'CaWO4': 128_000, 'Si': 648_000, 'Al203': 1_041_000, 'LiAlO2': None}  # mK
sommerfeld_constant = {'W': 1.01e3, 'Au': 0.729e3, 'Al': 1.356e3}  # pJ / mol / mK^2 
debye_frequency = {'CaWO4': 4.7e3, 'Si': 13.5e3, 'Al203': 21.7e3, 'LiAlO2': None}  # mK  # = kB*debye_temp/h
energy_gap = {'CaWO4': 5.2e-3, 'Si': 1.12e-3, 'Al203': 8.3e-3, 'LiAlO2': None}  # keV
molar_mass = {'W': None, 'Au': None, 'Al': None, 'CaWO4': None, 'Si': None, 'Al203': None, 'LiAlO2': None}  # g/mole
density = {'W': None, 'Au': None, 'Al': None, 'CaWO4': None, 'Si': None, 'Al203': None, 'LiAlO2': None}  # g/mm^3
electron_phonon_coupling = {'W': 2.1e-4}  # pW / mK^6 / mm^3 - multiply with T^5 (mK) 
thermal_conductivity = {'Au': 17.1e3}  # pW / mK^2 / mm - multiply with temperature (mK) and cross section (mm^2), divide by length (mm) 

# below are constants of nature

kB = 1.380649e-17  # mm ^ 2 * g / s^2 / mK
na = 6.02214076e23  # number of items per mole


## heat capacity absorber phonons

In [ ]:
Cph = 12 * np.pi ** 4 / 5 *   # 

## heat capacity TES electrons

## electron - phonon coupling

## bath coupling TES

## bath coupling absorber

## epsilon

## tau n

## pars dict

In [ ]:
pars = {
    'C': ,  # pJ / mK, is defined later bec mutable
    'Gb': np.array([5e-3, 5e-3]),  # pW / mK
    'G': np.array([[0., 1e-3], [1e-3, 0.], ]),  # heat cond between components, pW / mK
    'lamb': 0.003,  # thermalization time (s)
    'eps': np.array([[0.99, 0.01], [0.1, 0.9], ]),  # share thermalization in components
    'delta': np.array([[0.02, 0.98], ]),  # share thermalization in components
    'Rs': np.array([0.035]),  # Ohm
    'Rh': np.array([10]),  # Ohm
    'L': np.array([3.5e-7]),  # H
    'Rt0': np.array([0.2]),  # Ohm
    'k': np.array([2.]),  # 1/mK
    'Tc': np.array([15.]),  # mK
    'Ib': np.array([1.]),  # muA
    'dac': np.array([0.]),  # V
    'pulser_scale': np.array([1.]),  # scale factor
    'heater_attenuator': np.array([1.]),
    'tes_flag': np.array([True, False], dtype=bool),  # which component is a tes
    'heater_flag': np.array([False, True], dtype=bool),  # which component has a heater
    'i_sq': np.array([2 * 1e-12]),  # squid noise, A / sqrt(Hz)
    'tes_fluct': np.array([2e-4]),  # ??
    'emi': np.array([2e-10]),  # ??
    'Tb': None,  # function that takes one positional argument t, returns Tb
    'tau': np.array([10]),
}
